## Install packages and import libraries

In [1]:
# Install required Python packages
!pip install --upgrade pip
!pip install onnx onnxruntime git+https://github.com/espressif/esp-ppq.git
!pip install protobuf==3.20.2
!pip install torchvision==0.17.2
!git lfs install
!find imagenet-sample-images/ -mindepth 1 -name ".*" -exec rm -rf {} +
!pip install ultralytics


  Cloning https://github.com/espressif/esp-ppq.git to /private/var/folders/mv/96db_j8j3ng3t9pc8jczncdh0000gn/T/pip-req-build-zxajejaz
  Running command git clone --filter=blob:none --quiet https://github.com/espressif/esp-ppq.git /private/var/folders/mv/96db_j8j3ng3t9pc8jczncdh0000gn/T/pip-req-build-zxajejaz
  Resolved https://github.com/espressif/esp-ppq.git to commit 9445ddd436321189231c824f5b8e02b96e132bd0
  Preparing metadata (setup.py) ... done
Updated Git hooks.
Git LFS initialized.
find: imagenet-sample-images/: No such file or directory


In [2]:
from pathlib import Path
import os
from typing import Final
from PIL import Image
from ppq.api import espdl_quantize_onnx
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from ultralytics import YOLO
import sys
sys.path.append("./coco_detect/generate_onnx")
from export_onnx import ESP_YOLO, ESP_Attention, ESP_Detect
from ultralytics.nn.modules import Attention, Detect



    ___________ ____        ____  ____  ____ 
   / ____/ ___// __ \      / __ \/ __ \/ __ \
  / __/  \__ \/ /_/ /_____/ /_/ / /_/ / / / /
 / /___ ___/ / ____/_____/ ____/ ____/ /_/ / 
/_____//____/_/         /_/   /_/    \___\_\ 


Ultralytics 8.3.130 🚀 Python-3.10.11 torch-2.2.2 CPU (Apple M3)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from '../models/yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 64, 80, 80), (1, 80, 80, 80), (1, 64, 40, 40), (1, 80, 40, 40), (1, 64, 20, 20), (1, 80, 20, 20)) (5.4 MB)

ONNX: starting export with onnx 1.17.0 opset 13...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.5s, saved as '../models/yolo11n.onnx' (10.1 MB)

Export complete (0.7s)
Results saved to /Users/christophknaden/git/leezencounter/models
Predict:         yolo predict task=detect model=../models/yolo11n.onnx imgsz=640  
Validate:        yolo val task=detect model=../models/yolo11n.on

## Make predictions with base YOLO model

In [3]:
from pathlib import Path
from typing import Final

model_path = "./coco_detect/models/last.pt" # Add your model path here

model = YOLO(model_path)

torch_model = model.model.eval()

IMAGE_PATH: Final[Path] = Path('./calib_images_compressed') # Path to your images directory
OUTPUT_DIR: Final[Path] = Path('./preds_calib_images') # Path to save predictions
OUTPUT_DIR.mkdir(exist_ok=True)

for img_path in IMAGE_PATH.glob("*.jpg"):
    results = model(img_path)

    # Save the annotated image
    results[0].save(filename=OUTPUT_DIR / f"output_{img_path.name}")

    # Convert the results to a pandas DataFrame
    pred_df = results[0].to_df()

    # Save the DataFrame to CSV
    pred_df.to_csv(OUTPUT_DIR / f"output_{img_path.stem}.csv", index=False)



image 1/1 /Users/christophknaden/git/leezencounter/model-deployment/calib_images_compressed/frame226.jpg: 640x640 16 bicycles, 12 saddles, 68.2ms
Speed: 1.0ms preprocess, 68.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/christophknaden/git/leezencounter/model-deployment/calib_images_compressed/defisheye_frame_41_part1.jpg: 640x640 12 bicycles, 2 saddles, 51.9ms
Speed: 2.0ms preprocess, 51.9ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/christophknaden/git/leezencounter/model-deployment/calib_images_compressed/defisheye_frame_7_part1.jpg: 640x640 7 bicycles, 1 saddle, 62.7ms
Speed: 0.6ms preprocess, 62.7ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/christophknaden/git/leezencounter/model-deployment/calib_images_compressed/defisheye_frame_36_part1.jpg: 640x640 10 bicycles, 1 saddle, 50.6ms
Speed: 0.7ms preprocess, 50.6ms inference, 0.3ms postprocess per image at shape 

## Create .onnx file

In [4]:
model = ESP_YOLO(model_path)
for m in model.modules():
    if isinstance(m, Attention):
        m.forward = ESP_Attention.forward.__get__(m)
    if isinstance(m, Detect):
        m.forward = ESP_Detect.forward.__get__(m)

# Export to ONNX
model.export(format="onnx", simplify=True, opset=13, dynamic=True, imgsz=640)

Ultralytics 8.3.130 🚀 Python-3.10.11 torch-2.2.2 CPU (Apple M3)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'coco_detect/models/last.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 64, 80, 80), (1, 2, 80, 80), (1, 64, 40, 40), (1, 2, 40, 40), (1, 64, 20, 20), (1, 2, 20, 20)) (5.2 MB)

ONNX: starting export with onnx 1.17.0 opset 13...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.5s, saved as 'coco_detect/models/last.onnx' (9.9 MB)

Export complete (0.7s)
Results saved to /Users/christophknaden/git/leezencounter/model-deployment/coco_detect/models
Predict:         yolo predict task=detect model=coco_detect/models/last.onnx imgsz=640  
Validate:        yolo val task=detect model=coco_detect/models/last.onnx imgsz=640 data=./datasets/esp_camera/geoinformatics_institute/20250523/YOLO/geoinformatics_institute_yolo_dataset.yaml  
Visualize:       https://netron.app


'coco_detect/models/last.onnx'

## Create calibration dataset

In [5]:
DEVICE: str = 'cpu'

BATCH_SIZE = 1
IMAGE_SIZE = 640

IMAGENET_PATH: Final[Path] = Path('./calib_images_compressed') # Path to your calibration images directory

class ImageFolderDataset(Dataset):
    def __init__(self, image_dir: Path, transform=None):
        self.image_files = [
            image_dir / f for f in os.listdir(image_dir)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image = Image.open(self.image_files[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# Transformation (resize only — no normalization!)
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
])

# Create dataset and dataloader
cal_dataset = ImageFolderDataset(IMAGENET_PATH, transform=transform)
cal_dataloader = DataLoader(
    cal_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
)


## Create .espdl file

In [6]:
ONNX_YOLO_PATH: Final[Path] = Path('./coco_detect/generate_onnx/last.onnx') # Path to your ONNX model
ESPDL_YOLO_PATH: Final[Path] = Path('coco_detect/models/yolo11n.espdl') # Path to save the ESPDL model
TARGET_SOC: Final[str] = 'esp32s3'
NUM_OF_BITS: Final[int] = 8


x = cal_dataset[0]
if isinstance(x, (tuple, list)):
    x = x[0]
x = x.unsqueeze(0)

def collate_fn(batch: torch.Tensor) -> torch.Tensor:
    return batch.to(DEVICE)

# make use of ESP-PPQ to quantize the ONNX computation graph, optimize it for ESP32-S3 SoC, and convert it to .espdl
quantized_model = espdl_quantize_onnx(
    onnx_import_file=ONNX_YOLO_PATH.as_posix(),
    espdl_export_file=ESPDL_YOLO_PATH.as_posix(),
    calib_dataloader=cal_dataloader,
    calib_steps=8,
    input_shape=x.shape,
    inputs=None,
    target=TARGET_SOC,
    num_of_bits=NUM_OF_BITS,
    collate_fn=collate_fn,
    dispatching_override=None,
    device=DEVICE,
    error_report=True,
    skip_export=False,
    export_test_values=True,
    verbose=1,
)

[Warning] Unexpected input value of operation /model.11/Resize, recieving "None" at its input 1
[Warning] Unexpected input value of operation /model.14/Resize, recieving "None" at its input 1
[13:46:02] PPQ Quantization Fusion Pass Running ...       Finished.
[13:46:02] PPQ Quantize Simplify Pass Running ...         Finished.
[13:46:02] PPQ Parameter Quantization Pass Running ...    Finished.
[13:46:02] PPQ Runtime Calibration Pass Running ...       

Calibration Progress(Phase 2): 100%|██████████| 8/8 [00:01<00:00,  5.70it/s]


Finished.
[13:46:05] PPQ Quantization Alignment Pass Running ...    Finished.
[13:46:05] PPQ Passive Parameter Quantization Running ... Finished.
--------- Network Snapshot ---------
Num of Op:                    [296]
Num of Quantized Op:          [296]
Num of Variable:              [499]
Num of Quantized Var:         [499]
------- Quantization Snapshot ------
Num of Quant Config:          [915]
ACTIVATED:                    [334]
OVERLAPPED:                   [369]
PASSIVE:                      [194]
FP32:                         [18]
Network Quantization Finished.


Analysing Graphwise Quantization Error(Phrase 1):: 100%|██████████| 8/8 [00:01<00:00,  6.65it/s]
Analysing Graphwise Quantization Error(Phrase 2):: 100%|██████████| 8/8 [00:02<00:00,  3.11it/s]


Layer                                        | NOISE:SIGNAL POWER RATIO 
/model.10/m/m.0/ffn/ffn.1/conv/Conv:         | ████████████████████ | 28.559%
/model.10/m/m.0/attn/proj/conv/Conv:         | ███████████████████  | 26.769%
/model.23/cv3.2/cv3.2.1/cv3.2.1.1/conv/Conv: | █████████████        | 18.382%
/model.10/m/m.0/attn/MatMul_1:               | █████████████        | 18.097%
/model.22/m.0/cv2/conv/Conv:                 | ███████████          | 16.012%
/model.23/cv3.0/cv3.0.1/cv3.0.1.1/conv/Conv: | ███████████          | 15.402%
/model.23/cv3.2/cv3.2.1/cv3.2.1.0/conv/Conv: | ███████████          | 15.320%
/model.23/cv3.0/cv3.0.1/cv3.0.1.0/conv/Conv: | ██████████           | 14.280%
/model.10/m/m.0/attn/MatMul:                 | █████████            | 12.973%
/model.23/cv3.2/cv3.2.0/cv3.2.0.1/conv/Conv: | █████████            | 12.666%
/model.23/cv3.1/cv3.1.1/cv3.1.1.1/conv/Conv: | ████████             | 11.904%
/model.10/m/m.0/attn/qkv/conv/Conv:          | ████████             |

Analysing Layerwise quantization error:: 100%|██████████| 89/89 [01:59<00:00,  1.34s/it]


Layer                                        | NOISE:SIGNAL POWER RATIO 
/model.9/cv2/conv/Conv:                      | ████████████████████ | 0.140%
/model.1/conv/Conv:                          | ██████████████████   | 0.125%
/model.4/cv2/conv/Conv:                      | ████████             | 0.055%
/model.4/cv1/conv/Conv:                      | ████████             | 0.054%
/model.0/conv/Conv:                          | ███████              | 0.052%
/model.2/cv1/conv/Conv:                      | ████                 | 0.029%
/model.8/cv2/conv/Conv:                      | ██                   | 0.017%
/model.3/conv/Conv:                          | ██                   | 0.013%
/model.16/m.0/cv2/conv/Conv:                 | ██                   | 0.012%
/model.10/cv1/conv/Conv:                     | ██                   | 0.011%
/model.16/m.0/cv1/conv/Conv:                 | ██                   | 0.011%
/model.16/cv2/conv/Conv:                     | ██                   | 0.011%
/mo

# Helper Functions

## Compress images to 640x640

In [8]:
import os
from PIL import Image

# Input and output directories
input_dir = "./calib_images"
output_dir = "./calib_images_compressed"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Process all JPG files in the input directory
for filename in os.listdir(input_dir):
    if filename.lower().endswith('.jpg'):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        # Open, resize, and save the image
        with Image.open(input_path) as img:
            img_resized = img.resize((640, 640), Image.Resampling.LANCZOS)
            img_resized.save(output_path, quality=95)
            # Optional: use optimize flag for better compression
            # img_resized.save(output_path, optimize=True, quality=95)

print("Compression complete.")


Compression complete.


## Visualize predictions on image

In [9]:
import cv2

# Load image
image_path = "./calib_images_compressed/defisheye_frame_26_part1.jpg"
image = cv2.imread(image_path)

# List of predictions (category, score, x1, y1, x2, y2)
predictions = [
    (0, 0.531209, 86, 356, 138, 453),
    (0, 0.531209, 137, 363, 193, 461),
    (0, 0.407333, 408, 363, 470, 486),
    (0, 0.377541, 44, 350, 82, 424),
    (0, 0.377541, 129, 361, 171, 460),
    (0, 0.348645, 516, 188, 568, 294),
    (0, 0.348645, 168, 355, 232, 491),
    (0, 0.320821, 104, 361, 158, 457),
    (0, 0.245085, 67, 353, 124, 442),
    (0, 0.182426, 147, 357, 199, 467),
]


# Loop through predictions and draw boxes
for category, score, x1, y1, x2, y2 in predictions:
    label = f"Bicycle:{category} {score:.2f}"
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # green box
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Show image
#cv2.imshow("Detections", image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
# Or save it
cv2.imwrite("output_QAT.jpg", image)


True